In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 1000px; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))

import sys
import os

from IPython.display import display

sys.path.append(os.path.join(os.path.abspath(''), '..', '..'))

from tools.profiler import generate_schedule, parse_result, print_result

dlrm_perf_schedule = {
    # interested event name
    'BottomMLP.fc1': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'BottomMLP.fc2': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'BottomMLP.fc3': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'sparse_embedding1': {
        'forward_events': [
            'localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse'
        ],
        'backward_events': []
    },
    'interaction1': {
        'forward_events': [],
        'backward_events': [] 
    },
    'TopMLP.fc4': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc5': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc6': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc7': {
        'forward_events': [
            'fused_fully_connected.fprop',
            'fused_fully_connected.fprop.cublasGemmEx',
            'fused_fully_connected.fprop.add_bias_and_re_kernel',  
        ],
        'backward_events': [
            'fused_fully_connected.bprop',
            'fused_fully_connected.bprop.initialize_array',
            'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_fully_connected.bprop.convert_array',
            'fused_fully_connected.bprop.cublasGemmEx_1',
            'fused_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc8': {
        'forward_events': [],
        'backward_events': []
    },
}

In [ ]:
generate_schedule(dlrm_perf_schedule, repeat_for_each_event=50)

In [ ]:
#some how run the hugectr --train, not added yet

In [ ]:
result = parse_result()
print_result(result)

In [ ]:
import matplotlib.pyplot as plt
        
def timeline_chart(result):
    max_iter_time = 0.0
    for host in result:
        max_iter_time = max(max_iter_time, host["avg_iter_time_ms"])

    bar_height_value = 2
    bar_height_increase_value = 4

    each_bar_height_inches = 3
    figure_width_inches = max_iter_time * 160 / 10

    y_label_font_size = 24
    color_options = ['steelblue', 'paleturquoise', 'mediumorchid', 'yellowgreen', 'lightcoral', 'orange', 'chocolate', 'olive', 'darkseagreen']
    
    for host in result:
        stream_num = 0

        fig, ax = plt.subplots()
        current_height = 0
        y_labels = []
        y_ticks = []
        current_height += bar_height_increase_value

        for device_id, streams in host["timeline"].items():
            stream_num += len(streams)
            for stream_name, timeline in streams.items():
                event_timeline = []
                for event in timeline:
                    event_timeline.append([event["avg_iter_start_to_event_start_time_ms"], event["avg_measured_time_ms"]])
                ax.broken_barh(event_timeline, [current_height, bar_height_value], facecolors=color_options, alpha=0.8)
                y_ticks.append(current_height + float(bar_height_value) / 2)
                y_labels.append(device_id + '_' + stream_name)
                current_height += bar_height_increase_value

        y_ticks.append(current_height + float(bar_height_value) / 2)
        y_labels.append("Whole Iteration")
        ax.broken_barh([[0.0, max_iter_time]], [current_height, bar_height_value], facecolors=color_options, alpha=0.8)
        current_height += bar_height_increase_value

        fig.set_size_inches(figure_width_inches, each_bar_height_inches * (stream_num + 1))
        ax.set_xlim(0, max_iter_time)
        ax.grid(True)
        ax.set_yticks(y_ticks)
        ax.set_yticklabels(y_labels, fontdict={ 'fontsize' : y_label_font_size })

        ax.annotate('race interrupted', (61, 25),
                    xytext=(0.8, 0.9), textcoords='axes fraction',
                    arrowprops=dict(facecolor='black', shrink=0.05),
                    fontsize=16,
                    horizontalalignment='right', verticalalignment='top')

    plt.show()
        
timeline_chart(result)